# Install Necessary Libraries

In [ ]:
pip install openpyxl anonympy xlrd>=1.0.0

In [ ]:
pip install cape-privacy --no-deps

# Get File Meta Data

Enter the path of the data feeds

In [1]:
root = ''

Save a list of the feeds and their attributes to a CSV file and display the results.

In [2]:
import pandas as pd
import os

pd.set_option('display.max_rows', None)

full_path = []

for path, subdirs, files in os.walk(root):
    for name in files:
        if name != '.DS_Store' and '.asc' not in name:
            full_path.append(os.path.join(path, name))

df = pd.DataFrame(full_path, columns=['full_path'])
df = df['full_path'].str.split('/',expand=True)
df = df.drop([1,2,3,4], axis=1)
df = df.rename(columns={5:'object',6:'customer',7:'file'})
df['is_gpg'] = df['file'].str.contains('.gpg')
df['temp_file'] = df['file'].replace('.gpg','', regex=True)
df['dots'] = df['temp_file'].str.split('.').str.len()
df['file_type'] = df.apply(lambda x: x['temp_file'].split('.')[x['dots']-1].lower(), axis=1)
df = df.drop(['dots', 'temp_file'], axis=1)

df.to_csv('files.csv')

df_current = pd.read_excel('data_files.xlsx', sheet_name='files', header=0)

df_not_listed = df.merge(df_current.drop_duplicates(), on=['object','customer','file'], 
                   how='left', indicator=True)

df_not_listed[df_not_listed['_merge']=='left_only'][['object','customer','file','is_gpg','file_type_x']]

object            customer  \
5   Claims-Medical  ND-BCBS (Medicaid)   
6   Claims-Medical  ND-BCBS (Medicaid)   
7   Claims-Medical  ND-BCBS (Medicaid)   
19             ADT            Experian   
24      Carepointe                 All   
30      Carepointe                 All   
35       Claims-RX             ND BCBS   
46       Claims-RX         ND NextBlue   

                                                 file  is_gpg file_type_x  
5                        BCBSND_PRVGROUP_20230201.txt   False         txt  
6                         BCBSND_MED_CLM_20230201.txt   False         txt  
7                             BCBSND_PRV_20230201.txt   False         txt  
19  Updated_MemberMatch - Roster Sample File Layou...   False         txt  
24                                   revenue_pmpm.csv   False         csv  
30  quality-gap-report-by-patient+measure-cut-poin...   False         csv  
35                         BCBSND_RX_CLM_20230301.txt   False         txt  
46     NBND_2023_MA_Pharmacy_V7.12_20230220103007.txt   False         txt

In [3]:
import pandas as pd

# gsheet_name = "files"
# files = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={gsheet_name}"
# df_files = pd.read_csv(files)
# df_files = df_files.rename(columns={'Unnamed: 8':'sheet_id', 'Unnamed: 9':'header_row'})

df_files = pd.read_excel('data_files.xlsx', sheet_name='files', header=0)

# df_files

Extract all the fields from each feed and put them into a CSV file

In [ ]:
# get list of all fields in all files

import pandas as pd

data_dict = pd.DataFrame()

for f in [full_path[29]]:
    print(full_path[29])
    if '.DS_Store' not in f:
        # full_path = base_path + f
        f_split = f.split('/')
        object = f_split[5]
        customer = f_split[6]
        file = f_split[7]
        dots = len(file.split('.'))
        file_type = file.split('.')[dots-1]

        dd = pd.DataFrame()
        
        print(object, customer, file)

        if file_type.lower() == 'csv':
            df = pd.read_csv(f, nrows=10)
            fields = df.dtypes.to_frame('dtypes').reset_index()
            dd['field'] = fields['index']
            dd['type'] = fields['dtypes']
        if file_type.lower() == 'txt' and len(file.split('.')) < 3:
            df = pd.read_table(f, delimiter='|', nrows=10, encoding='latin_1')
            fields = df.dtypes.to_frame('dtypes').reset_index()
            dd['field'] = fields['index']
            dd['type'] = fields['dtypes']
        if file_type.lower() == 'xlsx':
            df = pd.read_excel(f, sheet_name=0, nrows=20, header=1)
            fields = df.dtypes.to_frame('dtypes').reset_index()
            dd['field'] = fields['index']
            dd['type'] = fields['dtypes']
        if file_type.lower() == 'xls':
            df = pd.read_excel(f, sheet_name=0, nrows=10, engine='xlrd')
            fields = df.dtypes.to_frame('dtypes').reset_index()
            dd['field'] = fields['index']
            dd['type'] = fields['dtypes']
            
        dd['object'] = object
        dd['customer'] = customer
        dd['file'] = file
        dd['file_type'] = file_type

        data_dict = pd.concat([data_dict, dd])

data_dict.to_csv('data_files.csv')

# Mask Data

Before masking the data, the csv files with feed names and feed columns must be loaded into a Google sheet so that masking rules can be defined.  These rules are based off of the masking functions available in the `anonympy` library.

This library will keep all the structure and rows of the feed files, but mask columns specified with masking rules.

Define the id of the Google Sheet and the desired location of the masked data.

In [ ]:
sheet_id = "1mrbbbpV3LAEVqewsyY89zFGnfIbohSr2y6zMxqwO4tI"
root = '/Users/anthonychamberas/Desktop/'

Functions to create directories if they don't exist and to mask data

In [4]:
import gpg_lite as gpg

def create_directory(path):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
        
def mask_data(df, date_fields, token_fields, fake_fields):
    # ensure data types are correct for masked fields
    df[date_fields] = df[date_fields].astype('datetime64[ns]')
    df[token_fields] = df[token_fields].astype(str)

    # mask data in current file per rules
    anonym = dfAnonymizer(df)

    anonym.categorical_fake(fake_fields)
    anonym.datetime_noise(date_fields)
    anonym.categorical_tokenization(token_fields, 12, key='bsNhFSaYSU')

    dfm = anonym.to_df()
    dfm = dfm[df.notna()]
    
    return dfm

Loop through each row feed list and apply respective masking rules in the list of columns

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
import json
import pandas as pd

from anonympy.pandas import dfAnonymizer 

sheet_name = "mapping_rules"
map_file = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_map_file = pd.read_csv(map_file)

sheet_name = "files"
files = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_files = pd.read_csv(files)

df_files = df_files.rename(columns={'Unnamed: 7':'sheet_id', 'Unnamed: 8':'header_row'})

rows = len(df_files.index)
start_row = 152
rows = 156

delimiters = {
    "comma":",",
    "pipe":"|",
    "tab":"\t"
}

for r in range(start_row, rows):
    
    object = df_files.iloc[r]['object']
    customer = df_files.iloc[r]['customer']
    parent_folder = df_files.iloc[r]['parent_folder']
    child_folder = df_files.iloc[r]['child_folder']
    file = df_files.iloc[r]['file']
    encr = df_files.iloc[r]['encr']
    file_type = df_files.iloc[r]['file_type']
    delimiter = df_files.iloc[r]['delimiter']
    sheet_id = df_files.iloc[r]['sheet_id'].astype(int)
    header_row = df_files.iloc[r]['header_row'].astype(int)
    skip = df_files.iloc[r]['skip']
    notes = df_files.iloc[r]['notes']
    
    file = file.replace('.gpg', '')
    f = 'Input_Files/'+object+'/'+customer+'/'+file
    output_path = root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+''
    
    if skip == 'yes':
        print('skipping ',file, 'due to', notes)
    else:
        print('start',file)

        # create sub folders if they don't exist and save results back to a file
        path = root + 'Output_Files'
        create_directory(path)
        path = root + 'Output_Files/'+parent_folder
        create_directory(path)
        path = root + 'Output_Files/'+parent_folder+'/'+child_folder
        create_directory(path)

        # get masking rules for current file
        dfm = df_map_file[df_map_file['customer'].eq(customer) & df_map_file['object'].eq(object) & df_map_file['file'].eq(file)]

        date_fields = dfm[dfm['mask method'].eq('datetime_noise')]['field']

        token_fields = dfm[dfm['mask method'].eq('categorical_tokenization')]['field']

        fake_fields = dfm[dfm['mask method'].eq('categorical_fake')][['field', 'mask type']]\
            .set_index('field')\
            .to_json(orient='columns')

        fake_fields = json.loads(fake_fields)['mask type']

        #read data from current file into dataframe

        if file_type.lower() == 'csv':
            df = pd.read_csv(f)
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_csv(root + '/Output_Files/'+mapped_customer+'/'+object+'/'+file+'', index=False)

        #### allow for tab delimited and fixed with files ####
        if file_type.lower() == 'txt' and delimiter == 'pipe' and len(file.split('.')) < 3:
            df = pd.read_table(f, delimiter='|', encoding='latin_1')
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_csv(root + '/Output_Files/'+mapped_customer+'/'+object+'/'+file+'', sep='|', index=False)

        if file_type.lower() == 'txt' and delimiter == 'tab' and len(file.split('.')) < 3:
            df = pd.read_table(f, delimiter='\t', encoding='latin_1')
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_csv(root + '/Output_Files/'+mapped_customer+'/'+object+'/'+file+'', sep='\t', index=False)

        #### mask excel files ####
        if file_type.lower() == 'xlsx':
            df = pd.read_excel(f, sheet_name=sheet_id, header=header_row)
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_excel(root + '/Output_Files/'+mapped_customer+'/'+object+'/'+file+'', index=False)
        if file_type.lower() == 'xls':
            df = pd.read_excel(f, sheet_name=sheet_id, header=header_row, engine='xlrd')
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_excel(root + '/Output_Files/'+mapped_customer+'/'+object+'/'+file+'x', index=False)

        print('end',file)

        

/Users/anthonychamberas/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


start BCBSND-ArkosHealth-Roster-08242023.txt sheet: nan
end BCBSND-ArkosHealth-Roster-08242023.txt sheet: nan
skipping  NDHIN-ArkosHealth-Roster-headers.txt due to nan
skipping  Roster Specifications Flat File Only v1.0 FINA... due to nan
start NV-Arkos-Roster-08252023.txt sheet: nan


/var/folders/xr/smm2095x7txfhj4s2x6wzqyw0000gn/T/ipykernel_14876/4012688009.py:45: RuntimeWarning: invalid value encountered in cast
  sheet_id = sid.astype(int) if df_files.iloc[r]['sheet_id'] else 0
/var/folders/xr/smm2095x7txfhj4s2x6wzqyw0000gn/T/ipykernel_14876/4012688009.py:48: RuntimeWarning: invalid value encountered in cast
  header_row = hr.astype(int) if hr else 0


end NV-Arkos-Roster-08252023.txt sheet: nan


# Tests and Expirements (Ignore)

In [7]:
# https://levelup.gitconnected.com/python-data-anonymization-masking-guide-de0b0aa0ca82

from anonympy.pandas.utils_pandas import available_methods
from anonympy.pandas.utils_pandas import fake_methods

print(available_methods())
print(fake_methods())

`numeric`:
        * Perturbation - "numeric_noise"
        * Binning - "numeric_binning"
        * PCA Masking - "numeric_masking"
        * Rounding - "numeric_rounding"

`categorical`:
        * Synthetic Data - "categorical_fake"
        * Synthetic Data Auto - "categorical_fake_auto"
        * Resampling from same Distribution - "categorical_resampling"
        * Tokenazation - "categorical_tokenization"
        * Email Masking - "categorical_email_masking"

`datetime`:
        * Synthetic Date - "datetime_fake"
        * Perturbation - "datetime_noise"

`general`:
        * Drop Column - "column_suppression"
        
None
A | aba, address, administrative_unit, am_pm, android_platform_token, ascii_company_email, ascii_email, ascii_free_email, ascii_safe_email
B | bank_country, bban, boolean, bothify, bs, building_number
C | cache_pattern, catch_phrase, century, chrome, city, city_prefix, city_suffix, color, color_name, company, company_email, company_suffix, coordinate, country, c

In [6]:
import pandas as pd
from anonympy.pandas import dfAnonymizer 

df1 = pd.DataFrame([{"id":"ABCDEF","sex":"M"},{"id":"ABCDEF","sex":"M"},{"id":"UVWXYZ","sex":"F"}])
df2 = pd.DataFrame([{"id":"ABCDEF"},{"id":"ABCDEF"},{"id":"GHIJKL"}])

anonym1 = dfAnonymizer(df1)
anonym1.categorical_tokenization('id', 12, key='testkey')
#anonym1.categorical_fake({'sex':'random_letter'})
anonym1.categorical_resampling('sex')

df1['masked_id'] = anonym1.to_df()['id']
df1['masked_sex'] = anonym1.to_df()['sex']
print(df1)
print('\n')

anonym2 = dfAnonymizer(df2)
anonym2.categorical_tokenization('id', 12, key='testkey')

df2['masked_id'] = anonym2.to_df()['id']
print(df2)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


       id sex     masked_id masked_sex
0  ABCDEF   M  50b95426ef40          M
1  ABCDEF   M  50b95426ef40          M
2  UVWXYZ   F  51dfcd54c153          M


       id     masked_id
0  ABCDEF  50b95426ef40
1  ABCDEF  50b95426ef40
2  GHIJKL  5dbeec07962e


In [ ]:
encrypted_file = '/Users/anthonychamberas/Desktop/Input_Files/Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg'
decrypt(encrypted_file)

gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg'
gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRV_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRV_20230201.txt'
gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt'
gpg --decrypt 'Claims-RX/ND BCBS/BCBSND_RX_CLM_20230301.txt.gpg' > 'Claims-RX/ND BCBS/BCBSND_RX_CLM_20230301.txt'
gpg --decrypt 'Claims-RX/ND NextBlue/NBND_2023_MA_Pharmacy_V7.12_20230220103007.txt.gpg' > 'Claims-RX/ND NextBlue/NBND_2023_MA_Pharmacy_V7.12_20230220103007.txt'


In [ ]:
import gpg_lite as gpg

gpg_store = gpg.GPGStore()

encrypted_file = '/Users/anthonychamberas/Downloads/Input_Files/Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt.gpg'
decrypted_file = encrypted_file[:-4]
with open(encrypted_file, "rb") as f, open(decrypted_file, "w") as f_out:
    gpg_store.decrypt(
      source=f,
      output=f_out,
      passphrase="Chuck Norris does not need one - the password needs him")

with open(decrypted_file, "r") as f:
    print("Decrypted message:", "".join(f.readlines()))

In [ ]:
def decrypt(encrypted_file):
    gpg_store = gpg.GPGStore()
    
    decrypted_file = encrypted_file[:-4]
    with open(encrypted_file, "rb") as f, open(decrypted_file, "w") as f_out:
        gpg_store.decrypt(
          source=f,
          output=f_out,
          passphrase="Chuck Norris does not need one - the password needs him")
        
    return ('decrypted', encrypted_file)
